In [1]:
!pip install google-api-python-client 

  Using cached google_api_python_client-2.166.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
Using cached google_api_python_client-2.166.0-py2.py3-none-any.whl (13.2 MB)
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)


In [21]:
from apiclient.discovery import build
import pandas as pd
import json
with open('secret.json') as f:
    secret = json.load(f)

In [ ]:
def video_search(youtube, q='Python 自動化', max_results=50):
    response = youtube.search().list(
        q=q,
        part="id,snippet",
        order='viewCount',
        type='video',
        maxResults=max_results
    ).execute() 

    items = response['items']
    items_id = []
    for item in items:
        item_id = {}
        item_id['video_id'] = item['id']['videoId']
        item_id['channel_id'] = item['snippet']['channelId']
        items_id.append(item_id)

    df_video = pd.DataFrame(items_id)

    return df_video


In [19]:
DEVELOPER_KEY  = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

df_video = video_search(youtube, q='Python 自動化', max_results=50)

In [26]:
max_results = 50


In [30]:
channel_ids = df_video['channel_id'].unique().tolist()

subscriber_list = youtube.channels().list(
    id=','.join(channel_ids),
    part='statistics',
    fields='items(id,statistics(subscriberCount))'
).execute()

In [37]:
subscribers = []
for item in subscriber_list['items']:
    subscriber = {}
    subscriber['channel_id'] = item['id']
    subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
    subscribers.append(subscriber)

df_subscribers = pd.DataFrame(subscribers)


In [39]:
df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
df_extracted = df[df['subscriber_count'] < 10000]
df_extracted

,video_id,channel_id,subscriber_count
41,8xFdQ-0t12s,UC4iUfGeYbYhAySUwCZge9Ig,1930
44,QdHd3hGMDAo,UCAXt2DZuGc8990EkQSPr8pg,5510
45,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,5980
47,eouKY0dQhFY,UCO4Cruq71rw9S9lPDjJVyDA,3460


In [46]:
video_ids = df_extracted['video_id'].tolist()
videos_list = youtube.videos().list(
    id=','.join(video_ids),
    part='snippet,statistics',
    fields='items(id,snippet(title),statistics(viewCount))'
).execute() 

In [ ]:
items = videos_list['items']
videos_info = []

for item in items:
    video_info = {}
    video_info['video_id'] = item['id']
    video_info['title'] = item['snippet']['title']
    video_info['view_count'] = item['statistics']['viewCount']
    videos_info.append(video_info)

videos_info
df_videos_info = pd.DataFrame(videos_info)

In [57]:
results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
results = results.loc[:, ['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

In [63]:
df_video = video_search(youtube, q='Python 自動化', max_results=50)

def get_results(df_video, threshold=5000):
    channel_ids = df_video['channel_id'].unique().tolist()
    subscriber_list = youtube.channels().list(
        id=','.join(channel_ids),
        part='statistics',
        fields='items(id,statistics(subscriberCount))'
    ).execute()

    subscribers = []
    for item in subscriber_list['items']:
        subscriber = {}
        if len(item['statistics']) > 0:
            subscriber['channel_id'] = item['id']
            subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
            subscribers.append(subscriber)
        else:
            subscriber['channel_id'] = item['id']
            
    df_subscribers = pd.DataFrame(subscribers)
    df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
    df_extracted = df[df['subscriber_count'] < threshold]

    video_ids = df_extracted['video_id'].tolist()
    videos_list = youtube.videos().list(
        id=','.join(video_ids),
        part='snippet,statistics',
        fields='items(id,snippet(title),statistics(viewCount))'
    ).execute() 

    items = videos_list['items']
    videos_info = []

    for item in items:
        video_info = {}
        video_info['video_id'] = item['id']
        video_info['title'] = item['snippet']['title']
        video_info['view_count'] = item['statistics']['viewCount']
        videos_info.append(video_info)

    df_videos_info = pd.DataFrame(videos_info)

    results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
    results = results.loc[:, ['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

    return results

In [64]:
df_video = video_search(youtube, q='Python Excel', max_results=50)
results =get_results(df_video, threshold=10000)